In [115]:
import os
import requests
import time
from io import BytesIO
from PIL import Image
from api_key import API_KEY

In [108]:
def get_images(street_name, street_number, town_name, file_name, output_directory='images', size='600x400', fov=120, api_key=API_KEY, angles=[0, 90, 180, 270]):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    geocoding_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={street_name} {street_number},{town_name}&key={api_key}'
    response = requests.get(geocoding_url)
    json_data = response.json()
    location = json_data['results'][0]['geometry']['location']
    lat, lng = location['lat'], location['lng']
    
    base_url = 'https://maps.googleapis.com/maps/api/streetview'
    
    for angle in angles:
        params = {
            'size': size,
            'location': f'{lat},{lng}',
            'fov': fov,
            'heading': angle,
            'key': api_key
        }
        response = requests.get(base_url, params=params)
        
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            output_file = os.path.join(output_directory, f'{file_name}_{angle}.jpg')
            image.save(output_file)
            print(f'Image saved as: {output_file}')
            if angle
        else:
            print(f'Error retrieving angle {angle}. Status {response.status_code}')


In [95]:
get_images('Rua Pedro Alvares Cabral', '800', 'Montes Claros')

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x400 at 0x7F73E976E700>
Image saved as: images/Rua Pedro Alvares Cabral_0.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x400 at 0x7F73EA23B280>
Image saved as: images/Rua Pedro Alvares Cabral_90.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x400 at 0x7F74081FE580>
Image saved as: images/Rua Pedro Alvares Cabral_180.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x400 at 0x7F73EA14C0A0>
Image saved as: images/Rua Pedro Alvares Cabral_270.jpg


In [122]:
def loop_get_images(street_name, town_name):
    street_number = 1
    prev_image = None
    
    while True:
        # Call the get_images function with current street number and file name
        file_name = f'{street_name}_{street_number}'
        get_images(street_name, street_number, town_name, file_name)
        
        # Check if the image of the first angle has changed from the previous iteration
        image_file = f"{file_name}_0.jpg"
        image_path = os.path.join("images", image_file)
        current_image = Image.open(image_path)
        
        if prev_image is not None and current_image.tobytes() == prev_image.tobytes():
            # Exit the loop if the image is the same as before
            break
        
        prev_image = current_image
        
        # Increment the street number by 50 for the next iteration
        time.sleep(5)
        street_number += 100


In [123]:
loop_get_images('Rua Pedro Alvares Cabral', 'Montes Claros')

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x400 at 0x7F73E9F86D90>
Image saved as: Rua Pedro Alvares Cabral_1/Rua Pedro Alvares Cabral_1_0.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x400 at 0x7F73E9F86D90>
Image saved as: Rua Pedro Alvares Cabral_1/Rua Pedro Alvares Cabral_1_90.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x400 at 0x7F73E9F8A370>
Image saved as: Rua Pedro Alvares Cabral_1/Rua Pedro Alvares Cabral_1_180.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x400 at 0x7F73E9F862E0>
Image saved as: Rua Pedro Alvares Cabral_1/Rua Pedro Alvares Cabral_1_270.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x400 at 0x7F73E9F86880>
Image saved as: Rua Pedro Alvares Cabral_101/Rua Pedro Alvares Cabral_101_0.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x400 at 0x7F73E9F8AFA0>
Image saved as: Rua Pedro Alvares Cabral_101/Rua Pedro Alvares Cabral_101_90.jpg
<PIL.JpegImagePlugin.JpegImageFile ima

FileNotFoundError: [Errno 2] No such file or directory: 'images/Rua Pedro Alvares Cabral_101_0.jpg'

In [136]:
def get_images(street_name, town_name, output_directory='images', size='600x400', fov=120, api_key=API_KEY, angles=[0, 90, 180, 270]):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    # Set initial values for previous_image
    previous_image = None
    current_image = 1
    
    # Start street number at 1
    street_number = 50
    
    while current_image != previous_image:
        # Construct geocoding URL with current street number
        geocoding_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={street_name} {street_number},{town_name}&key={api_key}'
        response = requests.get(geocoding_url)
        json_data = response.json()
        location = json_data['results'][0]['geometry']['location']
        lat, lng = location['lat'], location['lng']

        base_url = 'https://maps.googleapis.com/maps/api/streetview'

        for angle in angles:
            params = {
                'size': size,
                'location': f'{lat},{lng}',
                'fov': fov,
                'heading': angle,
                'key': api_key,
                'source': 'outdoor'
            }
            response = requests.get(base_url, params=params)

            if response.status_code == 200:
                image = Image.open(BytesIO(response.content))
                output_file = os.path.join(output_directory, f'{street_name}_{angle}_{street_number}.jpg')
                image.save(output_file)
                print(f'Image saved as: {output_file}')
                
                # Update previous and current images
                if angle == 0:
                    previous_image = current_image
                    current_image = image
                
            else:
                print(f'Error retrieving angle {angle}. Status {response.status_code}')

        # Increment street number by 50
        street_number += 100


In [ ]:
get_images('Rua Pedro Alvares Cabral', 'Montes Claros')

Image saved as: images/Rua Pedro Alvares Cabral_0_50.jpg
Image saved as: images/Rua Pedro Alvares Cabral_90_50.jpg
Image saved as: images/Rua Pedro Alvares Cabral_180_50.jpg
Image saved as: images/Rua Pedro Alvares Cabral_270_50.jpg
Image saved as: images/Rua Pedro Alvares Cabral_0_150.jpg
